<a href="https://colab.research.google.com/github/Leila828/instadeep_labs/blob/master/Fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/My Drive/dataz

/content/drive/My Drive/dataz


In [ ]:
ls

client_test.csv   invoice_test.csv   sample1.csv  sampleTest1.csv
client_train.csv  invoice_train.csv  sample.csv   sampleTest.csv


In [ ]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
np.random.seed(4590)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!pip install -U tf-hub-nightly
!pip install tfds-nightly
import tensorflow_hub as hub
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers

Requirement already up-to-date: tf-hub-nightly in /usr/local/lib/python3.6/dist-packages (0.10.0.dev202008220006)


In [ ]:
train_client=pd.read_csv('client_train.csv')
test_client=pd.read_csv('client_test.csv')
train_invoice=pd.read_csv('invoice_train.csv')
test_invoice=pd.read_csv('invoice_test.csv')
sub=pd.read_csv('sample1.csv')
train_client.head()


,disrict,client_id,client_catg,region,creation_date,target
0,60,train_Client_0,11,101,31/12/1994,0.0
1,69,train_Client_1,11,107,29/05/2002,0.0
2,62,train_Client_10,11,301,13/03/1986,0.0
3,69,train_Client_100,11,105,11/07/1996,0.0
4,62,train_Client_1000,11,303,14/10/2014,0.0


In [ ]:
sub.head()

,client_id,target
0,NaN,0.056900
1,NaN,0.102972
2,NaN,0.063485
3,NaN,0.037982
4,NaN,0.037217


In [ ]:
#Label encoding the counter_type
d={"ELEC":0,"GAZ":1}
train_invoice['counter_type']=train_invoice['counter_type'].map(d)

In [ ]:
train_client['client_catg'] = train_client['client_catg'].astype('object')
train_client['disrict'] = train_client['disrict'].astype('object')

test_client['client_catg'] = test_client['client_catg'].astype('object')
test_client['disrict'] = test_client['disrict'].astype('object')

In [ ]:
#changing the invoice date to datetime
for df in [train_invoice,test_invoice]:
    df['invoice_date'] = pd.to_datetime(df['invoice_date'])

In [ ]:
aggs={}
aggs['consommation_level_1'] = ['sum','max','min','mean']
aggs['consommation_level_2'] = ['sum','max','min','mean']
aggs['consommation_level_3'] =['sum','max','min','mean']
aggs['consommation_level_4'] = ['sum','max','min','mean']

In [ ]:
agg_trans = train_invoice.groupby(['client_id']).agg(aggs)
agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
agg_trans.reset_index(inplace=True)

df = (train_invoice.groupby('client_id')
          .size()
          .reset_index(name='{}transactions_count'.format('1')))

agg_trans = pd.merge(df, agg_trans, on='client_id', how='left')

In [ ]:
train = pd.merge(train_client,agg_trans, on='client_id', how='left')

In [ ]:
d={"ELEC":0,"GAZ":1}
test_invoice['counter_type']=test_invoice['counter_type'].map(d)

In [ ]:
agg_trans = test_invoice.groupby(['client_id']).agg(aggs)
agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
agg_trans.reset_index(inplace=True)

df = (test_invoice.groupby('client_id')
          .size()
          .reset_index(name='{}transactions_count'.format('1')))

agg_trans = pd.merge(df, agg_trans, on='client_id', how='left')

In [ ]:
test = pd.merge(test_client,agg_trans, on='client_id', how='left')

In [ ]:
for df in [train,test]:
    df['creation_date'] = pd.to_datetime(df['creation_date'])

In [ ]:
col_to_drop = ['client_id', 'creation_date']
for col in col_to_drop:
    if col in train.columns:
        train.drop([col], axis=1, inplace=True)
    if col in test.columns:
        test.drop([col], axis=1, inplace=True)

In [ ]:
from sklearn import preprocessing
for f in test.columns:
    if train[f].dtype=='object' or test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[f].values) + list(test[f].values))
        train[f] = lbl.transform(list(train[f].values))
        test[f] = lbl.transform(list(test[f].values))

In [ ]:
all_data_na = train.isnull().sum() 
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing ' :all_data_na})
missing_data.head(20)

,Missing


In [ ]:
target=train['target']
train.drop('target',axis=1,inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split  

X_tain, X_test, Y_train, Y_test = train_test_split(train, target, test_size=0.33, random_state=0)  

In [ ]:
X_tain.shape

(90780, 20)

In [ ]:
X_test.shape

(44713, 20)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# instantiate the model set max_depth to 6 , random_state to 0  and n_jobs=-1 
rf=RandomForestRegressor(max_depth=6, random_state=0,verbose=10,n_jobs=-1)
# fit X_train and y_train
rf=rf.fit(X_tain,Y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 100
building tree 2 of 100


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.5s


building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100


[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    1.0s


...

[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   19.8s


building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100


[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed:   22.8s


building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   24.3s finished


In [ ]:
# Let's predict the train and the test
rf_train_pred = rf.predict(X_tain)
rf_test_pred = rf.predict(X_test)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done  57 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done  81 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done  94 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks  

In [ ]:
# import xgboost as xgb 
import xgboost as xgb
xgb.DMatrix?

In [ ]:
#@title solution

# convert  train data and test data  to xgboost data structure using DMatrix please refer to the documentation 
# don't forgot to add  features names 
dtrain = xgb.DMatrix(data=X_tain,label=Y_train)
dtest = xgb.DMatrix(data=X_test,label=Y_test,feature_names=X_test.columns)
# specify parameters via map set the objective to regression loss function  please refer to the documentation  
param = {'max_depth':5, 'eta':0.1,'seed':0,
         'objective':'reg:squarederror' }
# set nbr of iteration to 100 
num_iter = 100
# fit  the model and check the requirements params
xgb_model=xgb.train(params=param, dtrain=dtrain,num_boost_round= num_iter,evals=[(dtrain,"train"),(dtest,"test")])

[0]	train-rmse:0.460497	test-rmse:0.460534
[1]	train-rmse:0.42579	test-rmse:0.42588
[2]	train-rmse:0.395414	test-rmse:0.395606
[3]	train-rmse:0.369	test-rmse:0.369231
[4]	train-rmse:0.346095	test-rmse:0.346425
[5]	train-rmse:0.326355	test-rmse:0.326779
[6]	train-rmse:0.309385	test-rmse:0.309917
[7]	train-rmse:0.294897	test-rmse:0.29555
[8]	train-rmse:0.28262	test-rmse:0.283384
[9]	train-rmse:0.272267	test-rmse:0.273141
[10]	train-rmse:0.263527	test-rmse:0.264511
[11]	train-rmse:0.256187	test-rmse:0.25728
[12]	train-rmse:0.250102	test-rmse:0.251317
[13]	train-rmse:0.245005	test-rmse:0.246352
[14]	train-rmse:0.240811	test-rmse:0.24223
[15]	train-rmse:0.23731	test-rmse:0.23886
[16]	train-rmse:0.234386	test-rmse:0.236073
[17]	train-rmse:0.232032	test-rmse:0.233795
[18]	train-rmse:0.230028	test-rmse:0.231955
[19]	train-rmse:0.228386	test-rmse:0.230418
[20]	train-rmse:0.227061	test-rmse:0.22918
[21]	train-rmse:0.225957	test-rmse:0.228171
[22]	train-rmse:0.225029	test-rmse:0.227311
[23]	train

In [ ]:
#@title solution
# Let's predict the train and the test
xgb_train_pred = xgb_model.predict(dtrain)
xgb_test_pred =  xgb_model.predict(dtest)

In [ ]:
import xgboost as xgb
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
clf = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=9,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=2019,
)
%time clf.fit(train, target)

CPU times: user 1min 53s, sys: 91.2 ms, total: 1min 53s
Wall time: 1min 53s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=None, n_estimators=300, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=2019,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, verbosity=1)

**Submission**

In [ ]:
submission = pd.DataFrame({"client_id": sub["client_id"],
        "target": xgb_test_pred[1]
    })
submission.to_csv('samplelast.csv', index=False)

In [ ]:
submission.head(10)

,client_id,target
0,test_Client_0,0.004138
1,test_Client_1,0.004138
2,test_Client_10,0.004138
3,test_Client_100,0.004138
4,test_Client_1000,0.004138
5,test_Client_10000,0.004138
6,test_Client_10001,0.004138
7,test_Client_10002,0.004138
8,test_Client_10003,0.004138
9,test_Client_10004,0.004138
